In [1]:
import pandas as pd
from collections import defaultdict
import math
import numpy as np
import string

In [2]:

punct = set(string.punctuation)


noun_suffix = ["action", "age", "ance", "cy", "dom", "ee", "ence", "er", "hood", "ion", "ism", "ist", "ity", "ling", "ment", "ness", "or", "ry", "scape", "ship", "ty"]
verb_suffix = ["ate", "ify", "ise", "ize"]
adj_suffix = ["able", "ese", "ful", "i", "ian", "ible", "ic", "ish", "ive", "less", "ly", "ous"]
adv_suffix = ["ward", "wards", "wise"]

In [3]:
def get_word_tag(line, vocab): 
    if not line.split():
        word = "--n--"
        tag = "--s--"
        return word, tag
    else:
        word, tag = line.split()
        if word not in vocab: 
            # Handle unknown words
            word = assign_unk(word)
        return word, tag
    return None 

In [4]:
def preprocess(vocab, data_fp):

    orig = []
    prep = []


    with open(data_fp, "r") as data_file:

        for cnt, word in enumerate(data_file):


            if not word.split():
                orig.append(word.strip())
                word = "--n--"
                prep.append(word)
                continue


            elif word.strip() not in vocab:
                orig.append(word.strip())
                word = assign_unk(word)
                prep.append(word)
                continue

            else:
                orig.append(word.strip())
                prep.append(word.strip())

    assert(len(orig) == len(open(data_fp, "r").readlines()))
    assert(len(prep) == len(open(data_fp, "r").readlines()))

    return orig, prep


In [5]:
def assign_unk(tok):

    # Digits
    if any(char.isdigit() for char in tok):
        return "--unk_digit--"

    # Punctuation
    elif any(char in punct for char in tok):
        return "--unk_punct--"

    # Upper-case
    elif any(char.isupper() for char in tok):
        return "--unk_upper--"

    # Nouns
    elif any(tok.endswith(suffix) for suffix in noun_suffix):
        return "--unk_noun--"

    # Verbs
    elif any(tok.endswith(suffix) for suffix in verb_suffix):
        return "--unk_verb--"

    # Adjectives
    elif any(tok.endswith(suffix) for suffix in adj_suffix):
        return "--unk_adj--"

    # Adverbs
    elif any(tok.endswith(suffix) for suffix in adv_suffix):
        return "--unk_adv--"

    return "--unk--"

In [6]:

with open("WSJ_02-21.txt", 'r') as f:
    training_corpus = f.readlines()

print(f"A few items of the training corpus list")
print(training_corpus[0:5])

A few items of the training corpus list
['In\tIN\n', 'an\tDT\n', 'Oct.\tNNP\n', '19\tCD\n', 'review\tNN\n']


In [7]:

with open("hmm_vocab.txt", 'r') as f:
    voc_l = f.read().split('\n')

print("A few items of the vocabulary list")
print(voc_l[0:50])
print()
print("A few items at the end of the vocabulary list")
print(voc_l[-50:])

A few items of the vocabulary list
['!', '#', '$', '%', '&', "'", "''", "'40s", "'60s", "'70s", "'80s", "'86", "'90s", "'N", "'S", "'d", "'em", "'ll", "'m", "'n'", "'re", "'s", "'til", "'ve", '(', ')', ',', '-', '--', '--n--', '--unk--', '--unk_adj--', '--unk_adv--', '--unk_digit--', '--unk_noun--', '--unk_punct--', '--unk_upper--', '--unk_verb--', '.', '...', '0.01', '0.0108', '0.02', '0.03', '0.05', '0.1', '0.10', '0.12', '0.13', '0.15']

A few items at the end of the vocabulary list
['yards', 'yardstick', 'year', 'year-ago', 'year-before', 'year-earlier', 'year-end', 'year-on-year', 'year-round', 'year-to-date', 'year-to-year', 'yearlong', 'yearly', 'years', 'yeast', 'yelled', 'yelling', 'yellow', 'yen', 'yes', 'yesterday', 'yet', 'yield', 'yielded', 'yielding', 'yields', 'you', 'young', 'younger', 'youngest', 'youngsters', 'your', 'yourself', 'youth', 'youthful', 'yuppie', 'yuppies', 'zero', 'zero-coupon', 'zeroing', 'zeros', 'zinc', 'zip', 'zombie', 'zone', 'zones', 'zoning', '{',

In [8]:
vocab = {} 

for i, word in enumerate(sorted(voc_l)): 
    vocab[word] = i       
    
print("Vocabulary dictionary, key is the word, value is a unique integer")
cnt = 0
for k,v in vocab.items():
    print(f"{k}:{v}")
    cnt += 1
    if cnt > 20:
        break

Vocabulary dictionary, key is the word, value is a unique integer
:0
!:1
#:2
$:3
%:4
&:5
':6
'':7
'40s:8
'60s:9
'70s:10
'80s:11
'86:12
'90s:13
'N:14
'S:15
'd:16
'em:17
'll:18
'm:19
'n':20


In [9]:

with open("WSJ_24.txt", 'r') as f:
    y = f.readlines()

print("A sample of the test corpus")
print(y[0:10])

A sample of the test corpus
['The\tDT\n', 'economy\tNN\n', "'s\tPOS\n", 'temperature\tNN\n', 'will\tMD\n', 'be\tVB\n', 'taken\tVBN\n', 'from\tIN\n', 'several\tJJ\n', 'vantage\tNN\n']


In [10]:
_, prep = preprocess(vocab, "test.words.txt")     

print('The length of the preprocessed test corpus: ', len(prep))
print('This is a sample of the test_corpus: ')
print(prep[0:10])

The length of the preprocessed test corpus:  34199
This is a sample of the test_corpus: 
['The', 'economy', "'s", 'temperature', 'will', 'be', 'taken', 'from', 'several', '--unk--']


# Part 1: Parts-of-speech tagging

In [11]:
def create_dictionaries(training_corpus, vocab):

    

    emission_counts = defaultdict(int)
    transition_counts = defaultdict(int)
    tag_counts = defaultdict(int)
    
    # Initialize "prev_tag" (previous tag) with the start state, denoted by '--s--'
    prev_tag = '--s--' 
    
    
    i = 0 
    

    for word_tag in training_corpus:
        

        i += 1
        

        if i % 50000 == 0:
            print(f"word count = {i}")
            
        
        word, tag = get_word_tag(word_tag,vocab) 
        
        
        transition_counts[(prev_tag, tag)] += 1
        
        
        emission_counts[(tag, word)] += 1

        
        tag_counts[tag] += 1

        prev_tag = tag
        
        
    return emission_counts, transition_counts, tag_counts

In [12]:
emission_counts, transition_counts, tag_counts = create_dictionaries(training_corpus, vocab)

word count = 50000
word count = 100000
word count = 150000
word count = 200000
word count = 250000
word count = 300000
word count = 350000
word count = 400000
word count = 450000
word count = 500000
word count = 550000
word count = 600000
word count = 650000
word count = 700000
word count = 750000
word count = 800000
word count = 850000
word count = 900000
word count = 950000


In [13]:
states = sorted(tag_counts.keys())
print(f"Number of POS tags (number of 'states'): {len(states)}")
print("View these POS tags (states)")
print(states)

Number of POS tags (number of 'states'): 46
View these POS tags (states)
['#', '$', "''", '(', ')', ',', '--s--', '.', ':', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB', '``']


In [14]:
print("transition examples: ")
for ex in list(transition_counts.items())[:3]:
    print(ex)
print()

print("emission examples: ")
for ex in list(emission_counts.items())[200:203]:
    print (ex)
print()

print("ambiguous word example: ")
for tup,cnt in emission_counts.items():
    if tup[1] == 'back': print (tup, cnt) 

transition examples: 
(('--s--', 'IN'), 5050)
(('IN', 'DT'), 32364)
(('DT', 'NNP'), 9044)

emission examples: 
(('DT', 'any'), 721)
(('NN', 'decrease'), 7)
(('NN', 'insider-trading'), 5)

ambiguous word example: 
('RB', 'back') 304
('VB', 'back') 20
('RP', 'back') 84
('JJ', 'back') 25
('NN', 'back') 29
('VBP', 'back') 4


In [15]:
def predict_pos(prep, y, emission_counts, vocab, states):

    num_correct = 0
    
    
    all_words = set(emission_counts.keys())
    

    total = len(y)
    for word, y_tup in zip(prep, y): 

       
        y_tup_l = y_tup.split()
        
        
        if len(y_tup_l) == 2:
            
            
            true_label = y_tup_l[1]

        else:
            
            continue
    
        count_final = 0
        pos_final = ''
        
       
        if word in vocab:
            for pos in states:

                        
                
                key = (pos,word)

               
                if key in emission_counts:

                    count = emission_counts[key]

                    if count>count_final: 

                        
                        count_final = count

                       
                        pos_final = pos

            
            if pos_final == true_label: 
            
               
                num_correct += 1
            
    
    accuracy = num_correct / total
    
    return accuracy


In [16]:
accuracy_predict_pos = predict_pos(prep, y, emission_counts, vocab, states)
print(f"Accuracy of prediction using predict_pos is {accuracy_predict_pos:.4f}")

Accuracy of prediction using predict_pos is 0.8889


# Part 2: Hidden Markov Models for POS

In [18]:
def create_transition_matrix(alpha, tag_counts, transition_counts):
  
    all_tags = sorted(tag_counts.keys())
    
    num_tags = len(all_tags)
    
    A = np.zeros((num_tags,num_tags))
    
    trans_keys = set(transition_counts.keys())
    
     
    
    
    for i in range(num_tags):
        
        
        for j in range(num_tags):

            
            count = 0
            key = (all_tags[i],all_tags[j])

            
            if key in transition_counts: 
                
                
                count = transition_counts[key]
                
            
            count_prev_tag = tag_counts[all_tags[i]]

            
            A[i,j] = (count + alpha) / (count_prev_tag + alpha*num_tags)

    return A


In [19]:
alpha = 0.001
for i in range(46):
    tag_counts.pop(i,None)

A = create_transition_matrix(alpha, tag_counts, transition_counts)

print(f"A at row 0, col 0: {A[0,0]:.9f}")
print(f"A at row 3, col 1: {A[3,1]:.4f}")


A_sub = pd.DataFrame(A[30:35,30:35], index=states[30:35], columns = states[30:35] )
print(A_sub)

A at row 0, col 0: 0.000007040
A at row 3, col 1: 0.1691
              RBS            RP           SYM        TO            UH
RBS  2.217069e-06  2.217069e-06  2.217069e-06  0.008870  2.217069e-06
RP   3.756509e-07  7.516775e-04  3.756509e-07  0.051089  3.756509e-07
SYM  1.722772e-05  1.722772e-05  1.722772e-05  0.000017  1.722772e-05
TO   4.477336e-05  4.472863e-08  4.472863e-08  0.000090  4.477336e-05
UH   1.030439e-05  1.030439e-05  1.030439e-05  0.061837  3.092348e-02


In [22]:
def create_emission_matrix(alpha, tag_counts, emission_counts, vocab):
   
    num_tags = len(tag_counts)
    
    
    all_tags = sorted(tag_counts.keys())
    
    
    num_words = len(vocab)
    
    
    
    B = np.zeros((num_tags, num_words))
    
    
    emis_keys = set(list(emission_counts.keys()))
    
   
    for i in range(num_tags): # complete this line
        
        for j in range(num_words): # complete this line

            count = 0
                    
            
            key = (all_tags[i],vocab[j])
            
            if key in emission_counts.keys(): # complete this line
        
                count = emission_counts[key]
                
           
            count_tag = tag_counts[all_tags[i]]
                
          
            B[i,j] = (count + alpha) / (count_tag+ alpha*num_words)

    return B

In [23]:
for i in range(46):
    tag_counts.pop(i,None)

B = create_emission_matrix(alpha, tag_counts, emission_counts, list(vocab))

print(f"View Matrix position at row 0, column 0: {B[0,0]:.9f}")
print(f"View Matrix position at row 3, column 1: {B[3,1]:.9f}")


cidx  = ['725','adroitly','engineers', 'promoted', 'synergy']


cols = [vocab[a] for a in cidx]


rvals =['CD','NN','NNS', 'VB','RB','RP']


rows = [states.index(a) for a in rvals]


B_sub = pd.DataFrame(B[np.ix_(rows,cols)], index=rvals, columns = cidx )
print(B_sub)

View Matrix position at row 0, column 0: 0.000006032
View Matrix position at row 3, column 1: 0.000000720
              725      adroitly     engineers      promoted       synergy
CD   8.201296e-05  2.732854e-08  2.732854e-08  2.732854e-08  2.732854e-08
NN   7.521128e-09  7.521128e-09  7.521128e-09  7.521128e-09  2.257091e-05
NNS  1.670013e-08  1.670013e-08  4.676203e-04  1.670013e-08  1.670013e-08
VB   3.779036e-08  3.779036e-08  3.779036e-08  3.779036e-08  3.779036e-08
RB   3.226454e-08  6.456135e-05  3.226454e-08  3.226454e-08  3.226454e-08
RP   3.723317e-07  3.723317e-07  3.723317e-07  3.723317e-07  3.723317e-07


# Part 3: Viterbi Algorithm and Dynamic Programming

In [24]:
def initialize(states, tag_counts, A, B, corpus, vocab):
    
    num_tags = len(tag_counts)
    
    
    best_probs = np.zeros((num_tags, len(corpus)))
    
    
    best_paths = np.zeros((num_tags, len(corpus)), dtype=int)
    
    
    s_idx = states.index("--s--")
    
    for i in range(num_tags): # complete this line
        
        
        if A[s_idx,i] == 0: # complete this line
            
            
            best_probs[i,0] = float('-inf')
        
        
        else:
           
            best_probs[i,0] = math.log(A[s_idx,i]) + math.log(B[i,vocab[corpus[0]]] )
                        
    
    return best_probs, best_paths


In [25]:
best_probs, best_paths = initialize(states, tag_counts, A, B, prep, vocab)

In [26]:

print(f"best_probs[0,0]: {best_probs[0,0]:.4f}") 
print(f"best_paths[2,3]: {best_paths[2,3]:.4f}")

best_probs[0,0]: -22.6098
best_paths[2,3]: 0.0000


In [28]:
def viterbi_forward(A, B, test_corpus, best_probs, best_paths, vocab):
   
    num_tags = best_probs.shape[0]
    for i in range(1, len(test_corpus)): 
        
        
        if i % 5000 == 0:
            print("Words processed: {:>8}".format(i))
            
        
        for j in range(num_tags):
            
            
            best_prob_i =  float("-inf")
            
           
            best_path_i = None

            
            for k in range(num_tags): 
            
                
                prob = best_probs[k,i-1]+math.log(A[k,j]) +math.log(B[j,vocab[test_corpus[i]]])
                
                if prob > best_prob_i: 
                    best_prob_i = prob
                    best_path_i = k
                    
            best_probs[j,i] = best_prob_i
            best_paths[j,i] = best_path_i

        
    return best_probs, best_paths


In [29]:
best_probs, best_paths = viterbi_forward(A, B, prep, best_probs, best_paths, vocab)

Words processed:     5000
Words processed:    10000
Words processed:    15000
Words processed:    20000
Words processed:    25000
Words processed:    30000


In [31]:
best_probs

array([[-2.26098263e+01, -2.47821563e+01, -3.40824650e+01, ...,
        -2.08553710e+05, -2.08571000e+05, -2.08575297e+05],
       [-2.30766065e+01, -2.45158390e+01, -3.50477430e+01, ...,
        -2.08553663e+05, -2.08566281e+05, -2.08579095e+05],
       [-2.35729882e+01, -2.99830506e+01, -3.19800466e+01, ...,
        -2.08559220e+05, -2.08565630e+05, -2.08564366e+05],
       ...,
       [-2.27555161e+01, -3.44006276e+01, -3.17437063e+01, ...,
        -2.08557240e+05, -2.08571146e+05, -2.08575443e+05],
       [-1.96637215e+01, -2.99165357e+01, -3.15812704e+01, ...,
        -2.08555283e+05, -2.08563618e+05, -2.08570959e+05],
       [-1.83628846e+01, -2.49885094e+01, -3.27766333e+01, ...,
        -2.08555245e+05, -2.08563741e+05, -2.08569258e+05]])

In [35]:
best_paths.shape

(46, 34199)

In [36]:
best_probs.shape

(46, 34199)

In [37]:
print(f"best_probs[0,1]: {best_probs[0,1]:.4f}") 
print(f"best_probs[0,4]: {best_probs[0,4]:.4f}") 

best_probs[0,1]: -24.7822
best_probs[0,4]: -49.5601


In [38]:
def viterbi_backward(best_probs, best_paths, corpus, states):
    
    m = best_paths.shape[1] 
    

    z = [None] * m
    
    
    num_tags = best_probs.shape[0]
    best_prob_for_last_word = float('-inf')
    
    
    pred = [None] * m
    
    for k in range(num_tags): 
       
        if best_probs[k,-1]>best_prob_for_last_word: 
            
            
            best_prob_for_last_word = best_probs[k,-1]

            z[m - 1]=k
            
  
    pred[m - 1] = states[k]

    for i in range(len(corpus)-1, -1, -1): 
        

        pos_tag_for_word_i = best_paths[np.argmax(best_probs[:,i]),i]

        z[i - 1] = best_paths[pos_tag_for_word_i,i]

        pred[i - 1] = states[pos_tag_for_word_i]

    return pred

In [39]:

pred = viterbi_backward(best_probs, best_paths, prep, states)
m=len(pred)
print('The prediction for pred[-7:m-1] is: \n', prep[-7:m-1], "\n", pred[-7:m-1], "\n")
print('The prediction for pred[0:8] is: \n', pred[0:7], "\n", prep[0:7])

The prediction for pred[-7:m-1] is: 
 ['see', 'them', 'here', 'with', 'us', '.'] 
 ['VB', 'PRP', 'RB', 'IN', 'PRP', '.'] 

The prediction for pred[0:8] is: 
 ['DT', 'NN', 'POS', 'NN', 'MD', 'VB', 'VBN'] 
 ['The', 'economy', "'s", 'temperature', 'will', 'be', 'taken']


## Predict On dataset

In [40]:
print('The third word is:', prep[3])
print('Your prediction is:', pred[3])
print('Your corresponding label y is: ', y[3])

The third word is: temperature
Your prediction is: NN
Your corresponding label y is:  temperature	NN



In [41]:
def compute_accuracy(pred, y):

    num_correct = 0
    total = 0
    
  
    for prediction, y in zip(pred, y):
        
        word_tag_tuple = y.split()
        
        
        
        if len(word_tag_tuple)!=2: # complete this line
            continue 
        
        word, tag = word_tag_tuple
        
        if prediction == tag: # complete this line
            num_correct += 1 
       
        total += 1
        
       
    return (num_correct/total)

In [42]:
print(f"Accuracy of the Viterbi algorithm is {compute_accuracy(pred, y):.4f}")

Accuracy of the Viterbi algorithm is 0.9528
